In [14]:
import numpy as np
import matplotlib.cm as cmap
import time
import os.path
import scipy
import pickle as pickle
from struct import unpack
from brian2 import *
import brian2
from brian2tools import *
from PIL import Image

In [15]:
def get_surrounding(coordinate, spread, length):
    y = coordinate[0]
    x = coordinate[1]
    x_min = 0 if (x-spread < 0) else (x-spread)
    x_max = (length-1) if (x+spread > length-1) else (x+spread)
    y_min = 0 if (y-spread < 0) else (y-spread)
    y_max = (length-1) if (y+spread > length-1) else (y+spread)
    
    coordinates = []
    for i in range(y_min, y_max+1):
        for j in range(x_min, x_max+1):
            if (i!=y or j!=x):
                coordinates.append((i,j))
    return coordinates

In [16]:
# initial random weights
dataPath = './initial_weights/'

length = 80
field_size = 2                     # 2/3/4/5/6

w_s = field_size*1               # simple cell width
l_s = field_size*2              # simple_cell_length

n_LGN = length*length
n_L4  = (length//(w_s*2))*(length//l_s)
n_L3  = n_L4
n_L2  = n_L3

weight = {}
weight['LGN_L4'] = 10
weight['L4_L4'] = 7*18
weight['L4_L3_round'] = 0.1*18
weight['L4_L3_direct'] = 5*18
weight['L3_L3'] = 7
weight['L3_L2'] = 7
weight['L2_L3'] = 7*18

In [17]:
# LGN ON >> L4 horizontal
# LGN OFF >> L4 horizontal
# feedforward, simple_cell_length*simple_cell_width to 1

weightList = np.random.random((n_LGN, n_L4))
weightList += weight['LGN_L4']

weights_on = []
weights_off = []

for i in range(0, length//(w_s*2)):                   # y axis
    for j in range(0, length//l_s):                   # x axis
        for p in range(0, w_s):
            for q in range(0, l_s):
                x_on = (i*2*w_s+p)*length + (j*l_s+q)
                x_off = ((i*2+1)*w_s+p)*length + (j*l_s+q)
                y = i*(length//l_s) + j
                
                weights_on.append((x_on, y, weightList[x_on,y]))
                weights_off.append((x_off, y, weightList[x_off,y]))
        
np.save(dataPath + 'LGN_L4_h_on', weights_on)
np.save(dataPath + 'LGN_L4_h_off', weights_off)

In [18]:
# LGN ON >> L4 vertical
# LGN OFF >> L4 vertical
# feedforward, simple_cell_length*simple_cell_width to 1

weightList = np.random.random((n_LGN, n_L4))
weightList += weight['LGN_L4']

weights_on = []
weights_off = []

for i in range(0, length//l_s):                   # y axis
    for j in range(0, length//(w_s*2)):          # x axis
        for p in range(0, l_s):
            for q in range(0, w_s):
                x_on = (i*l_s+p)*length + (j*2*w_s+q)
                x_off = (i*l_s+p)*length + ((j*2+1)*w_s+q)
                y = i*(length//(w_s*2)) + j
                
                weights_on.append((x_on, y, weightList[x_on,y]))
                weights_off.append((x_off, y, weightList[x_off,y]))
        
np.save(dataPath + 'LGN_L4_v_on', weights_on)
np.save(dataPath + 'LGN_L4_v_off', weights_off)

In [19]:
# LGN ON >> L4 diagonal (135 right)
# LGN OFF >> L4 diagonal (135 right)
# feedforward, simple_cell_length*simple_cell_width to 1

weightList = np.random.random((n_LGN, n_L4))
weightList += weight['LGN_L4']

weights_on = []
weights_off = []

for i in range(0, length//l_s):                   # y axis
    for j in range(0, length//l_s):               # x axis
        x_on_list = [
            (i*l_s+0)*length + (j*2*w_s+1), (i*l_s+0)*length + (j*2*w_s+2), (i*l_s+0)*length + (j*2*w_s+3),
            (i*l_s+1)*length + (j*2*w_s+2), (i*l_s+1)*length + (j*2*w_s+3),
            (i*l_s+2)*length + (j*2*w_s+3)
        ]
        x_off_list = [
            (i*l_s+1)*length + (j*2*w_s+0),
            (i*l_s+2)*length + (j*2*w_s+0), (i*l_s+2)*length + (j*2*w_s+1),
            (i*l_s+3)*length + (j*2*w_s+0), (i*l_s+3)*length + (j*2*w_s+1), (i*l_s+3)*length + (j*2*w_s+2)
        ]
        y = i*(length//(w_s*2)) + j
        
        for x_on in x_on_list:
            weights_on.append((x_on, y, weightList[x_on,y]))
            
        for x_off in x_off_list:
            weights_off.append((x_off, y, weightList[x_off,y]))
        
np.save(dataPath + 'LGN_L4_dr_on', weights_on)
np.save(dataPath + 'LGN_L4_dr_off', weights_off)

In [20]:
# LGN ON >> L4 diagonal (45 left)
# LGN OFF >> L4 diagonal (45 left)
# feedforward, simple_cell_length*simple_cell_width to 1

weightList = np.random.random((n_LGN, n_L4))
weightList += weight['LGN_L4']

weights_on = []
weights_off = []

for i in range(0, length//l_s):                   # y axis
    for j in range(0, length//l_s):           # x axis
        x_on_list = [
            (i*l_s+0)*length + (j*2*w_s+0), (i*l_s+0)*length + (j*2*w_s+1), (i*l_s+0)*length + (j*2*w_s+2),
            (i*l_s+1)*length + (j*2*w_s+0), (i*l_s+1)*length + (j*2*w_s+1),
            (i*l_s+2)*length + (j*2*w_s+0)
        ]
        x_off_list = [
            (i*l_s+1)*length + (j*2*w_s+3),
            (i*l_s+2)*length + (j*2*w_s+2), (i*l_s+2)*length + (j*2*w_s+3),
            (i*l_s+3)*length + (j*2*w_s+1), (i*l_s+3)*length + (j*2*w_s+2), (i*l_s+3)*length + (j*2*w_s+3)
        ]
        y = i*(length//(w_s*2)) + j
        
        for x_on in x_on_list:
            weights_on.append((x_on, y, weightList[x_on,y]))
            
        for x_off in x_off_list:
            weights_off.append((x_off, y, weightList[x_off,y]))
        
np.save(dataPath + 'LGN_L4_dl_on', weights_on)
np.save(dataPath + 'LGN_L4_dl_off', weights_off)

In [21]:
# L4 >> L4
# one to one

weightList = np.random.random(n_L4)
weightList += weight['L4_L4']
weights = [(i, i, weightList[i]) for i in range(n_L4)]
np.save(dataPath + 'L4_L4', weights)

In [22]:
# L4 >> L2/3 (L3)
# many to one (circle + direct)
# excitatory horizontal forward connections

weight_direct_list  = np.random.random(n_L4)
weight_direct_list += weight['L4_L3_direct']

weights = [(k, k, weight_direct_list[k]) for k in range(n_L4)]

# weights = []

weight_round_list   = np.random.random((n_L4, n_L3))
weight_round_list  += weight['L4_L3_round']

for i in range(0, length//l_s):                   # y axis
    for j in range(0, length//l_s):               # x axis
        w_length = (length//l_s)
        for p,q in get_surrounding((i,j), 1, w_length):
            source = p * w_length + q
            target = i * w_length + j
            weights.append((source, target, weight_round_list[source,target]))
                 
np.save(dataPath + 'L4_L3', weights)

In [23]:
# L2/3 (L3) >> L2/3 (L3)
# one to many (circle)
# excitatory lateral connections

weightList = np.random.random((n_L3, n_L3))
weightList += weight['L3_L3']

weights = []

for i in range(0, length//l_s):                   # y axis
    for j in range(0, length//l_s):               # x axis
        w_length = (length//l_s)
        for p,q in get_surrounding((i,j), 1, w_length):
            source = p * w_length + q
            target = i * w_length + j
            weights.append((source, target, weightList[source,target]))
        
np.save(dataPath + 'L3_L3', weights)

In [24]:
# L2/3 (L3) >> L2/3 inhibitory (L2)
# one to many (circle)
# excitatory lateral connections

weightList = np.random.random((n_L3, n_L2))
weightList += weight['L3_L2']

weights = []

for i in range(0, length//l_s):                   # y axis
    for j in range(0, length//l_s):               # x axis
        w_length = (length//l_s)
        for p,q in get_surrounding((i,j), 2, w_length):
            source = p * w_length + q
            target = i * w_length + j
            weights.append((source, target, weightList[source,target]))
        
np.save(dataPath + 'L3_L2', weights)

In [25]:
# L2/3 inhibitory (L2) >> L2/3 (L3)
# one to one 
# inhibitiry vertical downward connections

weightList = np.random.random(n_L2)
weightList += weight['L2_L3']
weights = [(i, i, weightList[i]) for i in range(n_L2)]
np.save(dataPath + 'L2_L3', weights)

In [13]:
get_surrounding((4,5), 1, 10)

[(3, 4), (3, 5), (3, 6), (4, 4), (4, 6), (5, 4), (5, 5), (5, 6)]